# Interactive minimal working example
In this notebook you are going to run a single simulation, interactively, using the `simulation module` in `dlensalot/ana/config/examples/`, and our goal is to plot the lensing potential estimate.

# Header

In [1]:
import os
from os.path import join as opj
import numpy as np
import healpy as hp

import matplotlib.pyplot as plt

import lenscarf
from lenscarf.run import run

cond4mpi does not exists
could not setup healpix hack. reverting to standard healpy
MSC: using scarf with 2 OMP threads


# Choose analysis

In [4]:
## minimal working example
fn = opj(os.path.dirname(os.path.abspath(lenscarf.__file__)),'ana/config/examples/mwe/conf_mwe_fullsky.py')
print(fn)
ana_mwe = run(config=fn, job_id='QE_lensrec', verbose=False).job

/global/u2/s/sebibel/git/lenscarf/lenscarf/ana/config/examples/mwe/conf_mwe_fullsky.py


# Check configuration
Before running lensing reconstruction, let us look into the configuration file

In [5]:
ana_mwe.__dict__.keys()

dict_keys(['dversion', 'tr', 'k', 'version', 'simidxs_mf', 'Nmf', 'TEMP_suffix', 'TEMP', 'lm_max_len', 'cls_unl', 'cls_len', 'Lmin', 'zbounds', 'zbounds_len', 'pb_ctr', 'pb_extent', 'lmax_ivf', 'sky_coverage', 'spectrum_type', 'lmin_teb', 'OBD', 'tpl', 'tpl_kwargs', 'lmin_tlm', 'lmin_elm', 'lmin_blm', 'nlev_t', 'nlev_p', 'simidxs', '_sims', 'sims', 'data_type', 'data_field', 'beam', 'nside', 'transferfunction', 'transf_tlm', 'transf_elm', 'transf_blm', 'ftl', 'fel', 'fbl', 'ftl_unl', 'fel_unl', 'fbl_unl', 'qe_tasks', 'QE_simidxs', 'qe_lm_max_qlm', 'qe_lmax_qlm', 'lmax_qlm', 'qe_filter_directional', 'filter', 'qlms_dd', 'cg_tol', 'chain_model', 'chain_descr', 'cl_analysis', 'btemplate_perturbative_lensremap', 'it_tasks', 'lm_max_unl', 'it_lm_max_qlm', 'lenjob_pbgeometry', 'lenjob_geometry', 'mf_dirname', 'it_simidxs', 'it_filter_directional', 'sims_MAP', 'itmax', 'iterator_typ', 'lensres', 'ffi', 'k_geom', 'mfvar', 'soltn_cond', 'stepper_model', 'cpp', 'dlensalot_model', 'overwrite_libd

In [6]:
ana_mwe.sims.__dict__.keys()

dict_keys(['pix_lib_phas', 'nlev_t', 'nlev_p', 'sims_cmb_len', 'cl_transf_T', 'cl_transf_P', 'nside'])

## And how does the simulation data look like?

In [8]:
ana_mwe.sims.sims.get_sim_pmap(0)

KeyboardInterrupt: 

# Run ananlysis

In [ ]:
ana_mwe.run()

CL[1]  18994092408985.168
GL[1] (before subtraction)  18994063198072.613
GL[1] (after subtraction)  -29210912.5546875
 [00:02:16] collecting bands > 00%
